# Test 1 - Clustering with 1 hard question
If you skip any pre processing please give a reason why

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import pandas as pd
import numpy as np

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# Set max rows and columns displayed in jupyter
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

#the following gives access to utils folder
#where utils package stores shared code
import os
import sys
PROJECT_ROOT = os.path.abspath(os.path.join(
                  os.getcwd(),
                  os.pardir)
)

#only add it once
if (PROJECT_ROOT not in sys.path):
    sys.path.append(PROJECT_ROOT)

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

In [ ]:
#Needed to find utils
PROJECT_ROOT=PROJECT_ROOT+"/.."
PROJECT_ROOT
if (PROJECT_ROOT not in sys.path):
    sys.path.append(PROJECT_ROOT)

In [ ]:
print(sys.path)

## Constants

In [ ]:
RANDOM_SEED=42

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.datasets import make_gaussian_quantiles
from sklearn.datasets import make_classification

X1, Y1 = make_gaussian_quantiles(n_features=2, n_classes=3)
# sns.scatterplot(x=X1[:,0], y=X1[:,1], hue=Y1)
# plt.scatter(X1[:, 0], X1[:, 1], marker="o", c=Y1, s=25, edgecolor="k")

## Make a dataset

In [ ]:
X1, Y1 = make_classification( class_sep=2.0, n_repeated=3,  n_features=10, n_redundant=2, n_informative=5, n_clusters_per_class=1, n_classes=5, random_state=42)
plt.figure(figsize=(20,10))
ax=sns.scatterplot(x=X1[:,0], y=X1[:,1], hue=Y1)

df=pd.DataFrame(data=X1, columns=('a','b','c','d','e','f','g','h','i','j'))

# df.describe()

#scale these hugely
df['a']=df['a']*1000
df['e']=df['e']*120

## Convert categoricals

In [ ]:
# !conda install -c anaconda sympy -y

#which are redundant
# from sympy import Matrix
# np.linalg.matrix_rank(X1)
# _, inds = Matrix(X1).rref(iszerofunc=lambda x: abs(x)<1e-3)
# inds
#d and G are redundant

#create a catagorical
# df['d']=pd.qcut(df['d'], 10,labels=['xxxxsmall','xxxsmall','xxsmall','xsmall','small', 'med','large','xlarge','xxlarge', 'xxxlarge'])
df['d']=pd.cut(df['d'], 10,labels=['xxxxsmall','xxxsmall','xxsmall','xsmall','small', 'med','large','xlarge','xxlarge', 'xxxlarge'])

# a=pd.cut(np.array([1, 7, 5, 4, 6, 3]), 3,labels=['bad','med','good'])
# /a

## Verify

In [ ]:
df.to_csv('data.csv', index=False)

df=pd.read_csv('data.csv')

In [ ]:
df.head()

df.info()

## Load Dataset

In [ ]:
df=pd.read_csv('data.csv')
df.head()

In [ ]:
df.describe()

# Test - Kmeans clustering - 90 points
<mark>(40 pts) For the above dataset, apply appropriate pre processing, standardization and dimensionality reduction. Please be sure to remove all redundant features.<br>
(40 pts) Cluster the data using Kmeans<br>
(10 pts) 3D scatterplot the clusters using cluster membership to assign point color

In [ ]:
df.dtypes

In [ ]:
df=pd.read_csv('data.csv')
df.head()

## Handle Cats

In [ ]:
#convert size
dic = {"xxxxsmall":0, "xxxsmall":1, "xxsmall":2, "xsmall":3, "small":4, "med":5, "large":6, "xlarge":7, "xxlarge":8, "xxxlarge":9}
# dic= {v:i for i,v in enumerate(df.d.unique().tolist())}  #this way changes the order
df.d=df.d.map(dic)

In [ ]:
df.d.value_counts()
dic

In [ ]:
df.head()

## Dump correlated features

In [ ]:
corr_matrix=df.corr()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

#to find pairs
to_drop=[(col,row) for col in upper.columns for row in upper[f'{col}'].index.tolist() if not pd.isnull(upper.loc[f'{row}',f'{col}']) and upper.loc[f'{row}',f'{col}']>0.95]

to_drop

In [ ]:
df.drop(['g','j','b'], axis=1, inplace=True)

## Standardize

In [ ]:
#valid submission
# from sklearn.preprocessing import StandardScaler
# df1= pd.DataFrame(StandardScaler().fit_transform(df), columns=df.columns)
# df1.describe()
# df1.head()

In [ ]:
import utils as ut
from sklearn.preprocessing import StandardScaler
df=(df.pipe(ut.scale,df.columns.tolist(),StandardScaler()))
df.describe()
df.head()

## PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=.95, whiten=True)
features_pca=pd.DataFrame(pca.fit_transform(df))
print(f'Orig #features={df.shape[1]}, number features containing 95% of variance={features_pca.shape[1]}')

# features_pca
pca.explained_variance_ratio_
sum(pca.explained_variance_ratio_)

## Cluster

### KMEANS- find numb clusters

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans

In [ ]:
# Instantiate the clustering model and visualizer
model = KMeans(random_state=RANDOM_SEED)
MIN_CLUSTERS=2
MAX_CLUSTERS=12
# plt.clf()
plt.close()
visualizer = KElbowVisualizer(model, k=(MIN_CLUSTERS,MAX_CLUSTERS))

a=visualizer.fit(features_pca)        # Fit the data to the visualizer

visualizer.show();        # Finalize and render the figure

### KMEANS assign cluster membership

In [ ]:
from sklearn.cluster import KMeans

# initialize with k-means++, this puts the initial cluster choices as far away from each other
# as possible which increases the chances for convergence
kmeans = KMeans(n_clusters=5, init='k-means++', random_state=RANDOM_SEED).fit(features_pca.iloc[:,[0,1,2,3]].to_numpy())

In [ ]:
df['cluster']=kmeans.labels_
# df

## Plot clusters

In [ ]:
colors1 = {-1:"brown",
            0:"cyan",
           1:"orange", 
           2:"purple",
           3:"green",
          4:"gray",
          5:"red",
          6:"blue"}
# %%capture
# importing required libraries
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
%matplotlib ipympl
plt.close()

# creating figure
fig = plt.figure(figsize=(20,10));

ax = Axes3D(fig,auto_add_to_figure=False);
ignore=ax.set_facecolor("white");
ignore=ax.grid(color="black");

ignore=fig.add_axes(ax,frameon=False)  #<Axes3D:>
colors=df['cluster'].map(colors1)
# creating the plot
ignore=ax.scatter(df.iloc[:,0], df.iloc[:,1], df.iloc[:,2], c=colors)  #<mpl_toolkits.mplot3d.art3d.Path3DCollection at 0x7f98ea314070>

# setting title and labels
ignore=ax.set_title("3D plot")
plt.show();
plt.figure().clear();
plt.close()

## Look at data

In [ ]:
# To_Plot = [ col for col in df.columns]
# plt.close()
# plt.figure()
# sns.pairplot(data=df[To_Plot])
# plt.show();

In [ ]:
df.var()

## (10 pts) Hard question (note that part 2 is extra credit, do it last):
for this dataset<br>
1. (4 pts) write a function to find the euclidian distance between any 2 points <br>
2. (4 pts-EXTRA CREDIT) given a point p1, write a function that will find the point p2 that is furthest from p1<br>
3. (3 pts) what is the complexity (Big O) of the above procedure<br>
4. (3 pts) what is the complexity (Big O) if you apply part 2 to every point in the dataset <br>

In [ ]:
#Q1
import math
def dst1(pt1,pt2):
    return(math.dist(pt1,pt2))

In [ ]:
#Q2 
# get a point to find max dist to
start_here=0
pt1=df.iloc[start_here,:]

# find the max dist
dsts={}
for i in range(len(df)):
    dsts[i]=dst1(pt1,df.iloc[i,:])

#get index and max distance
key=max(dsts, key=dsts.get)
print(f'The point furthest from {start_here} is pt[{key}] at a distance of {dsts[key]}')

In [ ]:
# Q3 O(n) compare 1 to all once
# Q4 O(n**2) compare all to all